In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.13.0
sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)
matplotlib 3.7.2
numpy 1.24.3
pandas 2.0.3
sklearn 1.3.0
tensorflow 2.13.0


In [7]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing(data_home='data')


In [8]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [9]:
!ls generate_csv

ls: cannot access 'generate_csv': No such file or directory


In [10]:
!rm -rf generate_csv


In [ ]:
type(x_train_scaled)


In [13]:
os.listdir()


['.idea',
 '1.py',
 '2023-7-19.ipynb',
 '23-7-18-1 练习.ipynb',
 '23-7-24.ipynb',
 '7.23-exercise.ipynb',
 '722练习.ipynb',
 'cal_housing_py3.pkz',
 'data',
 'exercise.py',
 'main.py',
 '练习2023-7-21.ipynb',
 '老师的练习.ipynb']

In [15]:
for file_idx, row_indices in enumerate(np.array_split(np.arange(24), 4)):
    print(file_idx,row_indices)


0 [0 1 2 3 4 5]
1 [ 6  7  8  9 10 11]
2 [12 13 14 15 16 17]
3 [18 19 20 21 22 23]


In [17]:
#当数据量特别大时，我们要考虑分多个csv存储，方便人工双击打开查看

#下面要把特征工程后的数据存为csv文件
output_dir = "generate_csv"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

#save_to_csv是工作可以直接复用的
def save_to_csv(output_dir, data, name_prefix,
                header=None, n_parts=10):
    #生成文件名 格式generate_csv/{}_{:02d}.csv
    path_format = os.path.join(output_dir, "{}_{:02d}.csv")
    filenames = []
    #把数据分为n_parts部分，写到文件中去
    for file_idx, row_indices in enumerate(
        np.array_split(np.arange(len(data)), n_parts)):
        #生成子文件名，用的字符串.format
        part_csv = path_format.format(name_prefix, file_idx)
        filenames.append(part_csv) #文件名添加到列表
        with open(part_csv, "w", encoding="utf-8") as f:
            #先写头部，如果头部不为空，头部是一个字符串
            if header is not None:
                f.write(header + "\n")
            for row_index in row_indices:
                #把字符串化后的每个字符串用逗号拼接起来
                f.write(",".join(
                    [repr(col) for col in data[row_index]]))
                f.write('\n')
    return filenames
#np.c_把x和y合并起来,按轴1合并
# np.c_等价于np.concatenate
train_data = np.c_[x_train_scaled, y_train]
valid_data = np.c_[x_valid_scaled, y_valid]
test_data = np.c_[x_test_scaled, y_test]
#头部,特征，也有目标
header_cols = housing.feature_names + ["MidianHouseValue"]
#把列表变为字符串
header_str = ",".join(header_cols)
print(header_str)
print('-'*50)
train_filenames = save_to_csv(output_dir, train_data, "train",
                              header_str, n_parts=20)
valid_filenames = save_to_csv(output_dir, valid_data, "valid",
                              header_str, n_parts=10)
test_filenames = save_to_csv(output_dir, test_data, "test",
                             header_str, n_parts=10)

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MidianHouseValue
--------------------------------------------------


In [19]:
temp_array=np.array([[1,2,3],[4,5,6]])
np.savetxt("temp.csv",temp_array)  #savetxt会自动将整型数或者浮点数转为字符串存储


In [20]:
!cat temp.csv

1.000000000000000000e+00 2.000000000000000000e+00 3.000000000000000000e+00
4.000000000000000000e+00 5.000000000000000000e+00 6.000000000000000000e+00


In [21]:
#看下生成文件的文件名
print(train_filenames)
import pprint  #为了打印美观性
print("train filenames:")
pprint.pprint(train_filenames)
print("valid filenames:")
pprint.pprint(valid_filenames)
print("test filenames:")
pprint.pprint(test_filenames)

['generate_csv\\train_00.csv', 'generate_csv\\train_01.csv', 'generate_csv\\train_02.csv', 'generate_csv\\train_03.csv', 'generate_csv\\train_04.csv', 'generate_csv\\train_05.csv', 'generate_csv\\train_06.csv', 'generate_csv\\train_07.csv', 'generate_csv\\train_08.csv', 'generate_csv\\train_09.csv', 'generate_csv\\train_10.csv', 'generate_csv\\train_11.csv', 'generate_csv\\train_12.csv', 'generate_csv\\train_13.csv', 'generate_csv\\train_14.csv', 'generate_csv\\train_15.csv', 'generate_csv\\train_16.csv', 'generate_csv\\train_17.csv', 'generate_csv\\train_18.csv', 'generate_csv\\train_19.csv']
train filenames:
['generate_csv\\train_00.csv',
 'generate_csv\\train_01.csv',
 'generate_csv\\train_02.csv',
 'generate_csv\\train_03.csv',
 'generate_csv\\train_04.csv',
 'generate_csv\\train_05.csv',
 'generate_csv\\train_06.csv',
 'generate_csv\\train_07.csv',
 'generate_csv\\train_08.csv',
 'generate_csv\\train_09.csv',
 'generate_csv\\train_10.csv',
 'generate_csv\\train_11.csv',
 'generate

In [22]:
filename_dataset = tf.data.Dataset.list_files(train_filenames)
for filename in filename_dataset:
    print(filename)


tf.Tensor(b'generate_csv\\train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_01.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_06.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_11.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_16.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\

In [23]:
filename_mydataset=tf.data.Dataset.from_tensor_slices(train_filenames)
filename_mydataset=filename_mydataset.repeat(1)
for i in filename_mydataset:
    print(i)

tf.Tensor(b'generate_csv\\train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_01.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_03.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_04.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_06.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_11.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\

In [24]:
#一访问list_files的dataset对象就随机了文件顺序
# for filename in filename_dataset:
#     print(filename)
n_readers = 5
dataset = filename_mydataset.interleave(
    #前面1行是header
#     lambda filename: tf.data.TextLineDataset(filename),
    #不带header,把特征名字去掉
    lambda filename: tf.data.TextLineDataset(filename).skip(1),
    cycle_length = n_readers,  #cycle_length和block_length增加获取了数据的随机性
    block_length=2
)
for line in dataset.take(15):
    print(line)

tf.Tensor(b'0.801544314532886,0.27216142415910205,-0.11624392696666119,-0.2023115137272354,-0.5430515742518128,-0.021039615516440048,-0.5897620622908205,-0.08241845654707416,3.226', shape=(), dtype=string)
tf.Tensor(b'-0.2980728090942217,0.3522616607867429,-0.10920507530549702,-0.25055520947444,-0.034064024638222286,-0.006034004264459185,1.080554840130013,-1.0611381656679573,1.514', shape=(), dtype=string)
tf.Tensor(b'0.8115083791797953,-0.04823952235146133,0.5187339067174729,-0.029386394873127775,-0.034064024638222286,-0.05081594842905086,-0.7157356834231196,0.9162751241885168,2.147', shape=(), dtype=string)
tf.Tensor(b'-0.6906143291679195,-0.1283397589791022,7.0201810347470595,5.624287386169439,-0.2663292879200034,-0.03662080416157129,-0.6457503383496215,1.2058962626018372,1.352', shape=(), dtype=string)
tf.Tensor(b'0.401276648075221,-0.9293421252555106,-0.05333050451405854,-0.1865945262276826,0.6545661895448709,0.026434465728210874,0.9312527706398824,-1.4406417263474771,2.512', shap

In [25]:
#parse csv 解析csv，通过decode_csv
# tf.io.decode_csv(str, record_defaults)

sample_str = '1,2,3,4,5'
record_defaults = [
    tf.constant(0, dtype=tf.int32),
    0,
    np.nan,
    "hello1",
    tf.constant([])#没有固定类型，默认是float32
]
# tf.io.decode_csv可以直接将字符串变为一个一个的tensor

#sample_str数据格式化，按照record_defaults进行处理
parsed_fields = tf.io.decode_csv(sample_str, record_defaults)
print(parsed_fields)

[<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(), dtype=string, numpy=b'4'>, <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]


In [26]:
#我们传一个空的字符串测试
#最后一个为1是可以转换的
try:
    parsed_fields = tf.io.decode_csv(',,,,1', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)
parsed_fields

[<tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'hello1'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0>]

In [29]:
#我们给的值过多的情况
try:
    parsed_fields = tf.io.decode_csv('1,2,3,4,5,6,', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

    #%%

#解析一行
def parse_csv_line(line, n_fields = 9):
    #先写一个默认的格式，就是9个nan,如果从csv中读取缺失数据，就会变为nan
    defs = [tf.constant(np.nan)] * n_fields
    #使用decode_csv解析
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    #前8个是x,特征，最后一个是y
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x, y

parse_csv_line(b'-0.9868720801669367,0.832863080552588,-0.18684708416901633,-0.14888949288707784,-0.4532302419670616,-0.11504995754593579,1.6730974284189664,-0.7465496877362412,1.138',
               n_fields=9)
def csv_reader_dataset(filenames, n_readers=5,
                       batch_size=32, n_parse_threads=5,
                       shuffle_buffer_size=10000):
    #把文件名类别变为dataset tensor
    dataset = tf.data.Dataset.list_files(filenames)
    #变为repeat dataset可以让读到最后一个样本时，从新去读第一个样本，可以无限循环
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        #skip(1)是因为每个文件存了特征名字，target名字
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size) #对数据进行洗牌，混乱
    #map，通过parse_csv_line对数据集进行映射，map只会给函数传递一个参数，这个参数
    #就是dataset中的tensor
    #n_parse_threads是代表是否开启了多线程
    dataset = dataset.map(parse_csv_line,
                          num_parallel_calls=n_parse_threads)
    #dataset分完batch后类型是 batch_dataset
    dataset = dataset.batch(batch_size)
    return dataset
#这里是一个测试,写4是为了大家理解
train_set = csv_reader_dataset(train_filenames, batch_size=4)
print(train_set)
print('-'*50)
i=0
#是csv_reader_dataset处理后的结果，
for x_batch, y_batch in train_set.take(2):
#     i=i+1
    print("x:")
    pprint.pprint(x_batch)
    print("y:")
    pprint.pprint(y_batch)
# print(i)

{{function_node __wrapped__DecodeCSV_OUT_TYPE_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expect 5 fields but have 7 in record 0 [Op:DecodeCSV]
<_BatchDataset element_spec=(TensorSpec(shape=(None, 8), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>
--------------------------------------------------
x:
<tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[ 0.15782312,  0.4323619 ,  0.3379948 , -0.01588031, -0.37338907,
        -0.05305246,  0.80061346, -1.2359096 ],
       [ 0.63034356,  1.8741661 , -0.06713215, -0.12543367, -0.19737554,
        -0.02272263, -0.69240725,  0.72652334],
       [-1.0591781 ,  1.3935647 , -0.02633197, -0.1100676 , -0.6138199 ,
        -0.09695935,  0.3247131 , -0.03747724],
       [-0.66722274, -0.04823952,  0.34529406,  0.53826684,  1.8521839 ,
        -0.06112538, -0.8417093 ,  1.5204847 ]], dtype=float32)>
y:
<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[3.169],
       [2.419],
       [0.67

In [30]:
%%time
batch_size = 32
train_set = csv_reader_dataset(train_filenames,
                               batch_size = batch_size)
valid_set = csv_reader_dataset(valid_filenames,
                               batch_size = batch_size)
test_set = csv_reader_dataset(test_filenames,
                              batch_size = batch_size)

CPU times: total: 109 ms
Wall time: 87.8 ms


In [31]:
#我们知道长度为8
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=[8]),
    keras.layers.Dense(1),
])
model.compile(loss="mean_squared_error", optimizer="sgd")
callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]

#当是BatchDataset,必须制定steps_per_epoch，validation_steps
history = model.fit(train_set,
                    validation_data = valid_set,
                    steps_per_epoch = 11160 // batch_size, #每epoch训练的次数，必须指定
                    validation_steps = 3870 // batch_size,
                    epochs = 100,
                    callbacks = callbacks)

Epoch 1/100
348/348 [==============================] - 1s 2ms/step - loss: 2.0126 - val_loss: 0.7938
Epoch 2/100
348/348 [==============================] - 0s 1ms/step - loss: 0.8954 - val_loss: 1.6227